In [20]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import lyricsgenius
import re

In [21]:
import os
path=os.getcwd()

In [24]:
# replace .. with client id and secret code from spotify developer (spotify web app)

cid="..."
secret="..."

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [29]:
playlist_link = " #  LINK OF A PLAYLIST "
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

# insert genius api account

genius = lyricsgenius.Genius("...", excluded_terms=["(Remix)", "(Live)"], 
                             timeout=60,
                retries=2, verbose=False)
genius.remove_section_headers = True

In [26]:
def get_track_info(playlist_track_uri):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    track_uri=[]
    track_name=[]
    track_artist=[]
    track_pop = []
    mode=[]
    duration=[]
    lyrics=[]
    genre=[]
    lyric_IO=[]

    pattern=[r'\bwith.', r'\bfeat.', r'\bft.', r'bremastered', r"/bRemastered"]

    control=0

    for track in sp.playlist_tracks(playlist_URI)["items"]:
        
        track_uri.append(track["track"]["uri"])
        song=track["track"]["name"]
        track_name.append(song)
        
        a=track["track"]["artists"]
        #artist=track["track"]["artists"][0]["name"]
        artist=a[0]["name"]
        track_artist.append(artist)

        genre.append(sp.artist(a[0]['uri'])["genres"])

        tmp=sp.audio_features(track["track"]['uri'])[0]

        danceability.append(tmp['danceability'])
        energy.append(tmp['energy'])
        mode.append(tmp['mode'])
        acousticness.append(tmp['acousticness'])
        instrumentalness.append(tmp['instrumentalness'])
        key.append(tmp['key'])
        loudness.append(tmp['loudness'])
        speechiness.append(tmp['speechiness'])
        liveness.append(tmp['liveness'])
        valence.append(tmp['valence'])
        tempo.append(tmp['tempo'])
        duration.append(tmp['duration_ms'])
        track_pop.append(track["track"]["popularity"])
        

        try:
            for elem in pattern:

                if re.search('-', song):
                    song=song.split("-", 1)[0]

                if re.search(elem, song)!=None:
                    p = r'\(.*?\)'
                    new_song = re.sub(p, '', song)
                    control=1
            
            print(control)

            if control==1:
                song_lyric = genius.search_song(title=new_song, artist=artist)
                if song_lyric==None:
                    lyrics.append('None')  
                    lyric_IO.append(0)
                else:
                    lyrics.append(song_lyric.lyrics)   
                    lyric_IO.append(1)
                print("--- {} ---".format(new_song))
            else:
                song_lyric = genius.search_song(title=song, artist=artist)
                if song_lyric==None:
                    lyric_IO.append(0)
                    lyrics.append('None')  
                else:
                    lyric_IO.append(1)
                    lyrics.append(song_lyric.lyrics)
                print("--- {} ---".format(song))


        except AttributeError:
            print("song {} of {} no lyrics available".format(song, artist))
            lyrics.append("None")
            lyric_IO.append(0)
        
        control=0

    df2 = pd.DataFrame({
    "track_uri":track_uri,
    "track_name":track_name,
    "track_artist":track_artist,
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo,
    "mode": mode,
    "duration": duration,
    'genre':genre,
    'lyrics_IO':lyric_IO,
    "popularity":track_pop})

    df3=pd.DataFrame({"track_uri":track_uri, "lyric":lyrics})

    
    return df2,df3

In [30]:
df1,df2=get_track_info(track_uris)

0
--- Belly Dancer ---
1
--- One Kiss  ---
0
--- No Lie ---
1
--- SexyBack  ---
1
--- Words  ---
1
--- International Love  ---
1
--- Head & Heart  ---
1
--- Heartbreak Anthem  ---
0
--- Dance The Night  ---
0
--- Five More Hours ---
1
--- Friday   ---
0
--- Let Me Love You ---
0
--- Feel So Close  ---
0
--- Don't Stop The Music ---
1
--- Whistle  ---
0
--- Temperature ---
0
--- CAN'T STOP THE FEELING! (from DreamWorks Animation's "TROLLS") ---
0
--- I Know You Want Me (Calle Ocho) ---
1
--- Lovers on the Sun  ---
0
--- Can't Get You out of My Head ---
1
--- Substitution  ---
0
--- Lose Control ---
0
--- The Nights ---
0
--- My Way ---
0
--- S&M ---
0
--- Levitating ---
0
--- All Nighter ---
0
--- Rush ---
1
--- Fireball  ---
0
--- I'm Good (Blue) ---
0
--- Mambo No. 5 (a Little Bit of...) ---
1
--- Instruction  ---
0
--- Rock This Party  ---
0
--- One in a Million ---
0
--- Blue (Da Ba Dee)  ---
1
--- Promises  ---
0
--- (It Goes Like) Nanana  ---
0
--- Lay Low ---
0
--- Single Ladies 

In [31]:
df1.to_csv(path+"playlist.csv")
df2.to_csv(path+"playlist_lyrics.csv")